In [8]:
import nutils as ntl
import numpy as np
import matplotlib.pyplot as plt
import inspect


## SUPG

We use SUPG(iNsErT mEtHoD hErE) to solve the advection dominated skew to mesh benchmark problem as described in [links go here]:

$$-\kappa \Delta u + \beta\cdot \nabla u = 0  $$

with $\kappa = 10^{-3}$, $\beta = (.5, 1)$ on a square grid with dirichlet boundary condition 

$$u = \begin{cases} 1 & y = 0 \text{ or } x = 0 \text{ and } y < 0.2 \\ 0 & \text{otherwise} \end{cases}   $$ 

In [113]:
degree = 1 #degree of basis functions
nelems = 9 #number of elements along edge of unit square
etype = 'square'

topo, geom =  ntl.mesh.unitsquare(nelems, etype) #rectilinear([np.linspace(0,1, nx), np.linspace(0,1, ny) ])
ns = ntl.function.Namespace()
ns.x = geom
ns.beta= np.array([0.5, 1.0])
ns.kappa = 1e-3
ns.basis = topo.basis('lagrange', degree = degree)
ns.u = 'basis_i ?dofs_i'

#element intrinsic time scale
ns.h = 0.5*np.sqrt(2/nelems**2) #mesh size parameter
ns.betanorm = 'beta_i beta_i'
ns.tau = 'h / (2 betanorm)'

#dirichlet boundary condition
    #pick out non-homogenous boundary
ns.badbnd = topo.boundary['left'].indicator(topo.boundary['left'][0:int(nelems/2)])
print(ns.badbnd)    
bezier = topo.sample('bezier', 2)
x = bezier.eval(ns.x)
badbnd1 = bezier.eval(ns.badbnd)
plt.plot(x, badbnd1)
    
sqr = topo.boundary['bottom'].integral('(u - 1)^2 d:x' @ ns, degree = degree*2)
sqr += topo.boundary['left'].integral( '(u - 1)^2 d:x' @ ns, degree = degree*2 )




#residual
res = topo.integral( 'kappa basis_i,j u_,j d:x' @ ns, degree = degree * 2)
res += topo.integral( 'tau beta_k basis_i,k (-kappa u_,jj + beta_n u_,n)  d:x' @ ns, degree = degree * 2) 

Array<>


EvaluationError: 
  %0 = EVALARGS
  %1 = Constant(9)()
  %2 = Tuple()
  %3 = SelectChain(evalargs=%0)
  %4 = TransformsIndexWithTail(trans=%3) --> ValueError: 

In [100]:
#print((beta *beta).sum(0)) #ct.getfullargspec(topo.basis))
#elp(np.dot)#(topo.basis)
help(topo.boundary['left'])
#print(topo.boundary)

Help on StructuredTopology in module nutils.topology object:

class StructuredTopology(Topology)
 |  StructuredTopology(root, axes, nrefine=0, bnames=(('left', 'right'), ('bottom', 'top'), ('front', 'back')))
 |  
 |  structured topology
 |  
 |  Method resolution order:
 |      StructuredTopology
 |      Topology
 |      nutils.types.Singleton
 |      nutils.types.Immutable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, root, axes, nrefine=0, bnames=(('left', 'right'), ('bottom', 'top'), ('front', 'back')))
 |      constructor
 |  
 |  __len__(self)
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      string representation
 |  
 |  basis_spline(self, degree, removedofs=None, knotvalues=None, knotmultiplicities=None, continuity=-1, periodic=None)
 |      spline basis
 |  
 |  basis_std(self, *args, **kwargs)
 |      bernstein shape functions
 |  
 |  getitem(self, item)
 |  
 |  locate(self, geom, coords, *, eps=0, **kwargs)
